In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
from darksky import forecast
from datetime import datetime as dt

from modules.nyc_shapefiles import read_nyc_shapefiles

In [2]:
precinct_dict, tract_dict, merged_census_info = read_nyc_shapefiles()

In [12]:
def get_precinct_centroids(precinct_dict):
    return {
        k: {
            'latitude': v['shape_gps'].centroid.y,
            'longitude': v['shape_gps'].centroid.x
        }
        for k, v in precinct_dict.items()
    }

In [29]:
def get_mean_latlon(centroids):
    arr = [[v['latitude'], v['longitude']] for v in centroids.values()]
    return np.mean(arr, axis=0)

In [38]:
nyc_mean_latlon = get_mean_latlon(centroids)

In [61]:
with open('../precrime_data/darksky_api_key') as f:
    darksky_api_key = f.read().replace('\n','')
    
t = dt(2017, 5, 13, 14).isoformat()
NYC = darksky_api_key, nyc_mean_latlon[0], nyc_mean_latlon[1]
with forecast(*NYC, time=t) as nyc_weather:
    for k, v in nyc_weather.currently._data.items():
        print('{0}:'.format(k), v)


time: 1494698400
summary: Rain
icon: rain
precipIntensity: 0.0945
precipProbability: 0.92
precipType: rain
temperature: 50.89
apparentTemperature: 50.89
dewPoint: 48.43
humidity: 0.91
pressure: 1010.78
windSpeed: 13.42
windBearing: 72
cloudCover: 1
uvIndex: 5
visibility: 2.97


In [62]:
def load_clean_felonies(data_path=None, clean_file=None):
    clean_file_defaults = {
        'data_path': '../precrime_data/',
        'clean_file': 'clean_felonies.csv',
        'dtype': {
            'CMPLNT_NUM': np.int64,
            'COMPLAINT_DATETIME': str,
            'REPORT_DATE': str,
            'KY_CD': np.int32,
            'OFNS_DESC': str,
            'BORO_NM': str,
            'ADDR_PCT_CD': np.int32,
            'Latitude': np.float64,
            'Longitude': np.float64,
        },
        'index_col': 'CMPLNT_NUM',
        'usecols': [
            'CMPLNT_NUM',
            'COMPLAINT_DATETIME',
            'REPORT_DATE',
            'KY_CD',
            'OFNS_DESC',
            'BORO_NM',
            'ADDR_PCT_CD',
            'Latitude',
            'Longitude',
        ],
        'parse_dates_cols': ['REPORT_DATE', 'COMPLAINT_DATETIME'],
    }
    
    if data_path is None:
        data_path = clean_file_defaults['data_path']
    if clean_file is None:
        clean_file = clean_file_defaults['clean_file']
        
    nypd_data = pd.read_csv(
        filepath_or_buffer = data_path+clean_file,
        index_col = clean_file_defaults['index_col'],
        usecols = clean_file_defaults['usecols'],
        dtype = clean_file_defaults['dtype'],
        parse_dates = clean_file_defaults['parse_dates_cols'],
        infer_datetime_format = True,
    )
    
    nypd_data.sort_values(by='COMPLAINT_DATETIME', inplace=True)
    return nypd_data

In [64]:
import pandas as pd
nypd_data = load_clean_felonies()

/Users/jsamet/anaconda2/envs/py3k/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [69]:
nypd_dates = nypd_data['COMPLAINT_DATETIME'].dt.date.unique()

In [76]:
hours = range(2,26,4)
for date in nypd_dates[:10]:
    for hour in hours:
        t = dt(date.year, date.month, date.day, hour).isoformat()
        print('Weather for: ', t)
        print('-----------')
        with forecast(*NYC, time=t) as nyc_weather:
            for k, v in nyc_weather.currently._data.items():
                print('{0}:'.format(k), v)
        print()


Weather for:  2006-01-02T02:00:00
-----------
time: 1136185200
summary: Clear
icon: clear-night
precipIntensity: 0
precipProbability: 0
temperature: 37.62
apparentTemperature: 34.61
dewPoint: 31.26
humidity: 0.78
pressure: 1023.6
windSpeed: 3.96
windBearing: 243
cloudCover: 0.24
uvIndex: 0
visibility: 9.35

Weather for:  2006-01-02T06:00:00
-----------
time: 1136199600
summary: Mostly Cloudy
icon: partly-cloudy-night
precipIntensity: 0
precipProbability: 0
temperature: 38.55
apparentTemperature: 35.32
dewPoint: 32.01
humidity: 0.77
pressure: 1023.18
windSpeed: 4.35
windBearing: 220
cloudCover: 0.84
uvIndex: 0
visibility: 9.52

Weather for:  2006-01-02T10:00:00
-----------
time: 1136214000
summary: Partly Cloudy
icon: partly-cloudy-day
precipIntensity: 0
precipProbability: 0
temperature: 41.55
apparentTemperature: 39.27
dewPoint: 33.17
humidity: 0.72
pressure: 1024.2
windSpeed: 3.82
windBearing: 184
cloudCover: 0.43
uvIndex: 1
visibility: 9.38

Weather for:  2006-01-02T14:00:00
--------

In [77]:
for hour in hours:
    print(hour)

2
6
10
14
18
22
